In [ ]:
from decimal import *
from scipy import signal
from scipy import fftpack
import wave
import csv
import gc
import numpy as np
import matplotlib.pyplot as plt

global smoothed_db
global magnification
global num

def to_db(x, N):
    pad = np.zeros(N//2)
    pad_data = np.concatenate([pad, x, pad])
    rms = np.array([np.sqrt((1/N) * (np.sum(pad_data[i:i+N]))**2) for i in range(len(x))])
    return 20 * np.log10(rms)

def smoothing(input, window):
    output = []
    for i in range(input.shape[0]):
        if i < window:
            output.append(np.mean(input[:i+window+1]))
        elif i > input.shape[0] - 1 - window:
            output.append(np.mean(input[i:]))
        else:
            output.append(np.mean(input[i-window:i+window+1]))
    return np.array(output)

def output_csv():
    global smoothed_db
    global magnification
    global num
    
    filename = './Lepton_dB/sum.csv'
    onesecsample = Decimal(len(smoothed_db)) / Decimal(60)
    onesample = Decimal(1) / Decimal(onesecsample)
    n = 3
    stopping = Decimal(0.000) + Decimal(magnification)
    
    with open(filename, 'a', newline="") as f:
        writer = csv.writer(f)
        for i in range (0, len(smoothed_db)):
            time = Decimal(onesample) * Decimal(i) + Decimal(magnification)
            rounddown = Decimal(str(time)).quantize(Decimal(str(10**(n*-1))), rounding=ROUND_DOWN)
            
            if rounddown == stopping:
                #if smoothed_db[i] == -float('inf') : 
                    #print(smoothed_db[i])
                    #continue (-infは無視)
                writer.writerow([rounddown,smoothed_db[i]])
                stopping += Decimal(0.001)
                stopping = Decimal(str(stopping)).quantize(Decimal(str(10**(n*-1))), rounding=ROUND_DOWN)
                
i = 0
while i <= 54:
    if i <= 9 :
        filename = "./Lepton_Audio/LeptonMic00" + str(i) + ".wav"
    else :
        filename = "./Lepton_Audio/LeptonMic0" + str(i) + ".wav"
    
    num = i
    magnification = 60 * i

    wave_file = wave.open(filename,"rb")
    x = wave_file.readframes(wave_file.getnframes())
    x = np.frombuffer(x, dtype= "int16")
    N = 1024 
    db = to_db(x, N)

    sr = 48000
    t = np.arange(0, db.shape[0]/sr, 1/sr)

    smoothed_db = smoothing(db, 100)
    output_csv()
    #1つのデータあたり0.00001041666秒

'''
    fig = plt.figure()

    fig = plt.figure(figsize=(8, 5))
    ax1 = fig.add_subplot(211)
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')

    ax1.plot(t, smoothed_db, label='signal', lw=1)
    ax1.legend()

    fig.tight_layout()

    plt.show()
    plt.close()
'''
    del db, smoothed_db, t
    gc.collect()
    
    i += 1

<ipython-input-1-311b4ae5c4a1>:18: RuntimeWarning: divide by zero encountered in log10
  return 20 * np.log10(rms)
